# Operaciones con datos en Python

## ¿Qué es un operador?

Los operadores son símbolos especiales en Python que realizan cálculos aritméticos o lógicos. Al valor sobre el que actúa el operador se le llama operando. Por ejemplo:

In [1]:
import networkx as nx
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransporteMasivoIA:
    def __init__(self):
        self.grafo = nx.DiGraph()
        self.estaciones_concurridas = set()
        self.estaciones_mantenimiento = set()
        self.model = self.crear_modelo()

    def agregar_estacion(self, nombre):
        self.grafo.add_node(nombre)

    def agregar_conexion(self, origen, destino, tiempo, concurrencia=0, transbordo=False):
        self.grafo.add_edge(origen, destino, tiempo=tiempo, concurrencia=concurrencia, transbordo=transbordo)

    def definir_estaciones_concurridas(self, estaciones):
        self.estaciones_concurridas.update(estaciones)

    def definir_estaciones_mantenimiento(self, estaciones):
        self.estaciones_mantenimiento.update(estaciones)

    def crear_modelo(self):
        model = keras.Sequential([
            layers.Dense(64, activation='relu', input_shape=(3,)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')
        ])
        model.compile(optimizer='adam', loss='mse')
        return model

    def entrenar_modelo(self, datos, etiquetas):
        datos_np = np.array(datos)
        etiquetas_np = np.array(etiquetas)
        self.model.fit(datos_np, etiquetas_np, epochs=100, verbose=0)

    def predecir_ruta(self, tiempo, concurrencia, transbordos):
        entrada = np.array([[tiempo, concurrencia, transbordos]])
        return self.model.predict(entrada)[0][0]

    def encontrar_ruta(self, origen, destino, preferencia):
        try:
            rutas = list(nx.all_simple_paths(self.grafo, origen, destino))
            mejor_ruta = None
            mejor_puntaje = float('inf')

            for ruta in rutas:
                tiempo_total = sum(self.grafo[u][v]['tiempo'] for u, v in zip(ruta, ruta[1:]))
                concurrencia_total = sum(self.grafo[u][v]['concurrencia'] for u, v in zip(ruta, ruta[1:]))
                transbordos_total = sum(1 for u, v in zip(ruta, ruta[1:]) if self.grafo[u][v]['transbordo'])

                # Ajuste de criterios según la preferencia del usuario
                if preferencia == "rapidez":
                    puntaje = tiempo_total
                elif preferencia == "menos_transbordos":
                    puntaje = transbordos_total
                elif preferencia == "menos_concurrida":
                    puntaje = concurrencia_total
                elif preferencia == "evitar_mantenimiento":
                    if any(est in self.estaciones_mantenimiento for est in ruta):
                        continue
                    puntaje = self.predecir_ruta(tiempo_total, concurrencia_total, transbordos_total)
                else:
                    puntaje = self.predecir_ruta(tiempo_total, concurrencia_total, transbordos_total)

                # Selección de la mejor ruta
                if puntaje < mejor_puntaje:
                    mejor_puntaje = puntaje
                    mejor_ruta = ruta

            return mejor_ruta if mejor_ruta else "No hay ruta disponible"
        except nx.NetworkXNoPath:
            return "No hay ruta disponible"
        except nx.NodeNotFound:
            return "Estación no encontrada"

# Creación del sistema de transporte
transporte = TransporteMasivoIA()

# Agregar estaciones
estaciones = ["A", "B", "C", "D", "E", "F", "G"]
for estacion in estaciones:
    transporte.agregar_estacion(estacion)

# Agregar conexiones entre estaciones
conexiones = [
    ("A", "B", 5, 10, False),
    ("B", "C", 7, 20, True),
    ("C", "D", 6, 5, False),
    ("D", "E", 8, 15, True),
    ("E", "F", 4, 30, False),
    ("F", "G", 3, 25, False)
]
for conexion in conexiones:
    transporte.agregar_conexion(*conexion)

# Definir estaciones concurridas y en mantenimiento
transporte.definir_estaciones_concurridas(["C", "E"])
transporte.definir_estaciones_mantenimiento(["D"])

# Simulación de datos de entrenamiento para la IA
datos_entrenamiento = [
    [5, 10, 0], [7, 20, 1], [6, 5, 0], [8, 15, 1], [4, 30, 0], [3, 25, 0]
]
etiquetas = [5, 7, 6, 8, 4, 3]  # Tiempos de llegada ideales según histórico
transporte.entrenar_modelo(datos_entrenamiento, etiquetas)

# Definir preferencias de los usuarios
usuarios = {
    "Viviana": "rapidez",
    "Diana": "menos_transbordos",
    "Andrea": "menos_concurrida",
    "Mauricio": "evitar_mantenimiento"
}

# Pedir al usuario su nombre
while True:
    nombre_usuario = input("Ingrese su nombre (Viviana, Diana, Andrea, Mauricio): ").strip()

    if nombre_usuario in usuarios:
        preferencia = usuarios[nombre_usuario]
        ruta = transporte.encontrar_ruta("A", "G", preferencia)
        print(f"Ruta óptima para {nombre_usuario} ({preferencia}): {ruta}")
        break
    else:
        print("Nombre no válido. Intente de nuevo.")


Exception ignored in: <function _xla_gc_callback at 0x7b0fdc60d800>
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
    
KeyboardInterrupt: 
/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Ingrese su nombre (Viviana, Diana, Andrea, Mauricio): Diana
Ruta óptima para Diana (menos_transbordos): ['A', 'B', 'C', 'D', 'E', 'F', 'G']


In [2]:
import networkx as nx
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

class TransporteMasivoIA:
    def __init__(self):
        self.grafo = nx.DiGraph()
        self.estaciones_concurridas = set()
        self.estaciones_mantenimiento = set()
        self.model = self.crear_modelo()

    def agregar_estacion(self, nombre):
        self.grafo.add_node(nombre)

    def agregar_conexion(self, origen, destino, tiempo, concurrencia=0, transbordo=False):
        self.grafo.add_edge(origen, destino, tiempo=tiempo, concurrencia=concurrencia, transbordo=transbordo)

    def definir_estaciones_concurridas(self, estaciones):
        self.estaciones_concurridas.update(estaciones)

    def definir_estaciones_mantenimiento(self, estaciones):
        self.estaciones_mantenimiento.update(estaciones)

    def crear_modelo(self):
        model = keras.Sequential([
            layers.Dense(64, activation='relu', input_shape=(3,)),
            layers.Dense(64, activation='relu'),
            layers.Dense(1, activation='linear')
        ])
        model.compile(optimizer='adam', loss='mse')
        return model

    def entrenar_modelo(self, datos, etiquetas):
        datos_np = np.array(datos)
        etiquetas_np = np.array(etiquetas)
        self.model.fit(datos_np, etiquetas_np, epochs=100, verbose=0)

    def predecir_ruta(self, tiempo, concurrencia, transbordos):
        entrada = np.array([[tiempo, concurrencia, transbordos]])
        return self.model.predict(entrada)[0][0]

    def encontrar_mejores_rutas(self, origen, destino, preferencia, max_rutas=3):
        try:
            rutas = list(nx.all_simple_paths(self.grafo, origen, destino))
            rutas_puntuadas = []

            for ruta in rutas:
                tiempo_total = sum(self.grafo[u][v]['tiempo'] for u, v in zip(ruta, ruta[1:]))
                concurrencia_total = sum(self.grafo[u][v]['concurrencia'] for u, v in zip(ruta, ruta[1:]))
                transbordos_total = sum(1 for u, v in zip(ruta, ruta[1:]) if self.grafo[u][v]['transbordo'])

                # Ajuste de criterios según la preferencia del usuario
                if preferencia == "rapidez":
                    puntaje = tiempo_total
                elif preferencia == "menos_transbordos":
                    puntaje = transbordos_total
                elif preferencia == "menos_concurrida":
                    puntaje = concurrencia_total
                elif preferencia == "evitar_mantenimiento":
                    if any(est in self.estaciones_mantenimiento for est in ruta):
                        continue
                    puntaje = self.predecir_ruta(tiempo_total, concurrencia_total, transbordos_total)
                else:
                    puntaje = self.predecir_ruta(tiempo_total, concurrencia_total, transbordos_total)

                rutas_puntuadas.append((ruta, puntaje))

            # Ordenar las rutas por el puntaje
            rutas_puntuadas.sort(key=lambda x: x[1])

            # Seleccionar las mejores rutas (máximo 3)
            mejores_rutas = [ruta for ruta, _ in rutas_puntuadas[:max_rutas]]
            return mejores_rutas if mejores_rutas else ["No hay rutas disponibles"]
        except nx.NetworkXNoPath:
            return ["No hay rutas disponibles"]
        except nx.NodeNotFound:
            return ["Estación no encontrada"]

# Creación del sistema de transporte
transporte = TransporteMasivoIA()

# Agregar estaciones
estaciones = ["A", "B", "C", "D", "E", "F", "G"]
for estacion in estaciones:
    transporte.agregar_estacion(estacion)

# Agregar conexiones entre estaciones
conexiones = [
    ("A", "B", 5, 10, False),
    ("B", "C", 7, 20, True),
    ("C", "D", 6, 5, False),
    ("D", "E", 8, 15, True),
    ("E", "F", 4, 30, False),
    ("F", "G", 3, 25, False),
    ("A", "C", 10, 15, False),
    ("B", "D", 8, 12, True),
    ("C", "E", 9, 18, True),
    ("D", "F", 7, 22, False)
]
for conexion in conexiones:
    transporte.agregar_conexion(*conexion)

# Definir estaciones concurridas y en mantenimiento
transporte.definir_estaciones_concurridas(["C", "E"])
transporte.definir_estaciones_mantenimiento(["D"])

# Simulación de datos de entrenamiento para la IA
datos_entrenamiento = [
    [5, 10, 0], [7, 20, 1], [6, 5, 0], [8, 15, 1], [4, 30, 0], [3, 25, 0]
]
etiquetas = [5, 7, 6, 8, 4, 3]  # Tiempos de llegada ideales según histórico
transporte.entrenar_modelo(datos_entrenamiento, etiquetas)

# Definir preferencias de los usuarios
usuarios = {
    "Viviana": "rapidez",
    "Diana": "menos_transbordos",
    "Andrea": "menos_concurrida",
    "Mauricio": "evitar_mantenimiento"
}

# Pedir al usuario su nombre
while True:
    nombre_usuario = input("Ingrese su nombre (Viviana, Diana, Andrea, Mauricio): ").strip()

    if nombre_usuario in usuarios:
        preferencia = usuarios[nombre_usuario]
        rutas = transporte.encontrar_mejores_rutas("A", "G", preferencia, max_rutas=3)

        print(f"\nRutas óptimas para {nombre_usuario} ({preferencia}):")
        for i, ruta in enumerate(rutas, 1):
            print(f"Ruta {i}: {ruta}")
        break
    else:
        print("Nombre no válido. Intente de nuevo.")


Ingrese su nombre (Viviana, Diana, Andrea, Mauricio): Diana

Rutas óptimas para Diana (menos_transbordos):
Ruta 1: ['A', 'C', 'D', 'F', 'G']
Ruta 2: ['A', 'B', 'C', 'D', 'F', 'G']
Ruta 3: ['A', 'B', 'D', 'F', 'G']


In [ ]:
print(2 + 5)

7


Aquí, `+` es el operador que realiza la suma. `2` y `5` son los operandos y `7` es el resultado de la operación. Ahora veamos con qué operadores contamos en cada tipo de datos:

## Operadores aritméticos

Los operadores aritméticos se utilizan para realizar operaciones matemáticas como sumas, restas, multiplicaciones, etc. Estos son compatibles con cualquier tipo de dato numérico, enteros, flotantes o complejos. En la siguiente tabla veremos los principales operadores aritméticos que tiene python:

| Operador | Descripción   | Ejemplo |
|------|------|------|
|+ |Suma |` r = 3 + 2`  |
|- |Resta |` r = 4 - 7 ` |
|- |Negación |` r = -7`  |
|* |Multiplicación |` r = 2 * 6`  |
|** |Exponente |` r = 2 ** 6`  |
|/ |División|`  r = 3.5 / 2`  |
|// |División entera|`  r = 3.5 // 2`  |
|% |Módulo |` r = 7 % 2 ` |

Si bien la suma, resta, multiplicación y las potencias tienen el comportamiento que hemos aprendido desde las matemáticas, como vemos, python tiene 3 operaciones relacionadas con la división, ahora, vamos a ver un ejemplo de uso de todas las operaciones y entrarémos en detalle del comportamiento de estas últimas tres operaciones.



### Suma
Ejecuta el siguiente bloque de código insertando diferentes tipos de número:

In [ ]:
a = eval(input("digita un número"))
b = eval(input("digita otro número"))
print("la suma es:", a + b)

digita un número412
digita otro número35
la suma es: 447


### Resta
Ejecuta el siguiente bloque de código insertando diferentes tipos de número:

In [ ]:
a = eval(input("digita un número"))
b = eval(input("digita otro número"))
print("la resta es:", a - b)

digita un número 845
digita otro número 165


la resta es: 680


### Multiplicación
Ejecuta el siguiente bloque de código insertando diferentes tipos de número:

In [ ]:
a = eval(input("digita un número"))
b = eval(input("digita otro número"))
print("la multiplicación es:", a * b)

digita un número 658
digita otro número 3.14


la multiplicación es: 2066.12


### Potenciación
Ejecuta el siguiente bloque de código insertando diferentes tipos de número:

In [ ]:
a = eval(input("digita un número"))
b = eval(input("digita otro número"))
print("la potencia a^b es:", a ** b)

digita un número 2
digita otro número 6


la potencia a^b es: 12


### División
<br>
<br>
<center><img src="https://www.jpvalencia.com/static/6be4f984ce532788d16a73f020891d92/213b2/division.png" alt="división" style="width:300px;"/></center>
<br>
<br>
<br>

Como se mencionó anteriormente, python cuenta con 3 operaciones relacionadas con la división: la división entera, la división completa y el módulo.

La **división entera** arroja el cociente de la operación, independientemente de que los números sean enteros o flotantes y este resultado ignora la parte decimal que pudiera tener el cociente. Su operador es `//`


In [ ]:
print('5//2: ', 5 // 2)
print('4525.214//2: ', 4525.214 // 2)
print('54874136.85//65874.6: ', 54874136.85 // 65874.6)

5//2:  2
4525.214//2:  2262.0
54874136.85//65874.6:  833.0


La **división completa** es la operación tradicional de la mayoría de lenguajes de programación, su cociente tiene en cuenta la fracción decimal que sea necesaria para hacer cero el residuo. Su operador es `/`

In [ ]:
print('5/2: ', 5 / 2)
print('4525.214/2: ', 4525.214 / 2)
print('54874136.85/65874.6: ', 54874136.85 / 65874.6)

5/2:  2.5
4525.214/2:  2262.607
54874136.85/65874.6:  833.0090330719275


La operación de **módulo** retorna el residuo de la división entre dos números y su operador es `%`. Si bien esta operación puede realizarse con flotantes, no es una práctica recomendada, ya que si usamos operandos flotantes obtendremos un valor de punto flotante como cociente y el resto será cero, lo que no generará una respuesta correcta.

Así que si usamos operandos flotantes en el operador de módulo, siempre obtendremos la respuesta 0, pero como veremos en el ejemplo, esto no siempre ocurre en Python. Si queremos obtener la respuesta correcta de módulo debemos usar operandos enteros

In [ ]:
print('5%2: ', 5 % 2)
print('4525.214%2.5: ', 4525.214 % 2.5)

5%2:  1
4525.214%2.5:  0.2139999999999418


## Operadores de comparación
Los operadores de comparación se utilizan para comparar valores. Devuelven `True` o `False` según la condición.

| operador | Descripción   | Ejemplo |
|------|------|------|
|==|a = b|` 5 == 3` |
|!=|  a diferente de b|  ` 5 != 3`  |
|< | a menor que b| `  5 < 3`  |
|> | a mayor que b| `  5 > 3`  |
|<= | a menor o igual que b|`   5 <= 3`  |
|>= | a mayor o igual que b|  ` 5 >= 3 ` |

In [ ]:
x = 69
y = 420

print('x > y es', x > y)
print('x < y es', x < y)
print('x == y es', x == y)
print('x != y es', x != y)
print('x >= y es', x >= y)
print('x <= y es', x <= y)

x > y es False
x < y es True
x == y es False
x != y es True
x >= y es False
x <= y es True


## Operadores lógicos
Permiten realizar operaciones de [álgebra booleana](https://plato.stanford.edu/entries/boolalg-math/), opera directamente sobre los valores de `True` y `False` y retorna uno de estos mismos valores.

| operador | Descripción                               | Ejemplo         |
|----------|:-----------------------------------------:|-----------------|
|and       |a y b                                      |`a and b`        |
|or        | a o b                                     |` a or b`        |
|not       | negación de a                             | ` not a `       |
|is        |Verdadero si los operandos son idénticos   |` x is True `    |
|is not    |Verdadero si los operandos no son idénticos|` x is not True` |

In [ ]:
x = True
y = False
x1 = 5
y1 = 5
x2 = 'Hello'
y2 = 'Hello'

print('x and y es', x and y)
print('x or y es', x or y)
print('not x es', not x)
print(x1 is not y1)
print(x2 is y2)



x and y es False
x or y es True
not x es False
False
True


## Operadores de asignación
Los operadores de asignación se utilizan en Python para asignar valores a las variables.

`a = 5` es un operador de asignación simple que asigna el valor $5$ de la derecha a la variable `a` de la izquierda.

Hay varios operadores compuestos en Python como `a += 5` que suma a la variable y luego asigna lo mismo. Es equivalente a `a = a + 5`. Es importante aclarar que debe existir de antemano la variable y tener un valor internamente para que estos operadores funcionen, de lo contrario se obtendrá el siguiente error:

In [ ]:
l += 5

NameError: name 'l' is not defined

A continuación se presenta una tabla con los principales operadores de asignación que podríamos utilizar.

| Operador   | Ejemplo  | Equivalente a   |
| ---------- | -------- | --------------- |
| =        | `x = 5 ` | `x = 5`         |
| +=       | `x += 5` | `x = x + 5`     |
| -=       | `x -= 5` | `x = x - 5`     |
| *=       | `x *= 5`| `x = x * 5`    |
| /=       | `x /= 5` | `x = x / 5`     |
| %=       |`x %= 5`  | `x = x % 5`     |
| //=      |`x //= 5` | `x = x // 5`    |



In [ ]:
x = 123456
print('1: ', x)
x += 333
print('2: ', x)
x *= 2
print('3: ', x)
x -= 555
print('4: ', x)
x /= 5
print('5: ', x)

1:  123456
2:  123789
3:  247578
4:  247023
5:  49404.6
